## Data Preprocessing

Before building a recommendation model, we must clean and prepare the dataset:
- Normalise ratings from a **0-5 scale** to a **0-1 scale**.
- Filter out movies without an IMDb ID to align with MovieLens data.
- Ensure all required data is correctly formatted.

In [26]:
import os
import pandas as pd

script_dir = os.getcwd()

print(f"Current working directory: {script_dir}")

Current working directory: c:\Users\willi\Documents\GitHub\Movie-Recommendation-System


## Matching IMDb Movies with MovieLens Dataset

In [27]:
# File paths
links_file = os.path.join(script_dir, "MovieLens Datasets", "links.csv")
merged_file = os.path.join(script_dir, "Cleaned Datasets", "merged_movie_data.tsv")

# Load datasets
df_links = pd.read_csv(links_file)
df_merged = pd.read_csv(merged_file, sep="\t", dtype=str, na_values="\\N")

# Drop tmdbId column
df_links = df_links.drop(columns=["tmdbId"])

# Drop NaN values
df_links = df_links.dropna(subset=['imdbId'])

# Number of data rows 
print(f"Total data rows in MovieLens database - links.csv: {len(df_links)}")
print(f"Total data rows in filtered IMDB database - merged_movie_data.tsv: {len(df_merged)}")

# Links datafile 
display(df_links.head(10))

Total data rows in MovieLens database - links.csv: 86537
Total data rows in filtered IMDB database - merged_movie_data.tsv: 47840


,movieId,imdbId
0,1,114709
1,2,113497
2,3,113228
3,4,114885
4,5,113041
5,6,113277
6,7,114319
7,8,112302
8,9,114576
9,10,113189


MovieLens dataset (`links.csv`) has fewer entries compared to the IMDB dataset (`merged_movie_data.tsv`). Therefore, we will remove the rows in the IMDB dataset where the IMDB ID does not have a corresponding MovieLens ID, based on the links.csv dataset.

We will do this by:
- Extracting and formatting IMDb IDs from the links.csv file.
- Matching these IDs with the tconst column in the merged IMDb dataset.

In [28]:
# Create a set of formatted IMDb IDs
movielens_imdb_ids = set(df_links['imdbId'].apply(lambda x: f"tt{int(x):07d}"))

# Filter the IMDb dataset using the set of IMDb IDs
filtered_imdb = df_merged[df_merged['tconst'].isin(movielens_imdb_ids)]

print(f"Total IMDb movies matched with MovieLens: {len(filtered_imdb)}")
    
display(filtered_imdb.head(5))


Total IMDb movies matched with MovieLens: 38178


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers
0,tt0000574,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,nm0846879,nm0846879
1,tt0002130,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy",7.1,"nm0078205,nm0655824,nm0209738",nm0019604
2,tt0002423,Passion,0,1919,113,"Biography,Drama,Romance",6.7,nm0523932,"nm0266183,nm0473134"
3,tt0002844,Fantômas: In the Shadow of the Guillotine,0,1913,54,"Crime,Drama",6.9,nm0275421,"nm0019855,nm0275421,nm0816232"
4,tt0003014,Ingeborg Holm,0,1913,96,Drama,7.0,nm0803705,"nm0472236,nm0803705"


In [29]:
# Load tags datafile
tags_file = os.path.join(script_dir, "MovieLens Datasets", "tags.csv")
df_tags = pd.read_csv(tags_file)

print("Sample of tags file:")
display(df_tags.head(5))

Sample of tags file:


,userId,movieId,tag,timestamp
0,10,260,good vs evil,1430666558
1,10,260,Harrison Ford,1430666505
2,10,260,sci-fi,1430666538
3,14,1221,Al Pacino,1311600756
4,14,1221,mafia,1311600746


### Processing Movie Tags and Mapping to IMDb IDs

-  **Group tags by `movieId`**: Aggregate tags into comma-separated lowercase lists, ensuring uniqueness.
- **Merge with IMDb links**: Join with `df_links` to retrieve `imdbId` for each `movieId`.
- **Filter relevant columns**: Retain only `imdbId` and aggregated `tags`.
- **Display results**: Print a sample and the total number of rows.

In [30]:
# Group tags by movieId and aggregate into comma-separated lists
tags_grouped = df_tags.groupby('movieId')['tag'].apply(lambda x: ', '.join(x.dropna().str.lower().unique())).reset_index()
tags_grouped.rename(columns={'tag': 'tags'}, inplace=True)

# display(tags_grouped.head(5))

# Merge tags with links to get IMDb IDs
tags_grouped = tags_grouped.merge(df_links, on="movieId", how="inner")

# Keep only relevant columns 
df_tags_filtered = tags_grouped[["imdbId", "tags"]]

# Display
print("Sample of grouped tags with IMDB IDs:")
display(df_tags_filtered.head(5))

print(f"Total data rows in tags file: {len(df_tags_filtered)}")

Sample of grouped tags with IMDB IDs:


,imdbId,tags
0,114709,"animation, friendship, toys, disney, pixar, cg..."
1,113497,"animals, based on a book, fantasy, magic board..."
2,113228,"sequel, moldy, old, old age, old men, wedding,..."
3,114885,"characters, chick flick, girl movie, revenge, ..."
4,113041,"family, pregnancy, wedding, 4th wall, aging, b..."


Total data rows in tags file: 53452


Now I  need to merge these tags with my filtered_imdb_movies

In [31]:
# Convert IMDb IDs to formatted strings directly in the DataFrame
df_links['tconst'] = df_links['imdbId'].apply(lambda x: f"tt{int(x):07d}")

# Create a mapping DataFrame
id_mapping = df_links[['movieId', 'tconst']]

# Merge the tags with the mapping
tags_with_imdb = pd.merge(tags_grouped, id_mapping, on='movieId', how='inner')

# Merge with the filtered IMDb dataset
result = pd.merge(filtered_imdb, tags_with_imdb[['tconst', 'tags']], on='tconst', how='left')

# Fill NaN tags with an empty string
result['tags'] = result['tags'].fillna('')

# Save the result
output_file = os.path.join(script_dir, "Cleaned Datasets", "Final_Movie_Data.tsv")
result.to_csv(output_file, sep="\t", index=False)

# Print and display the result
print(f"Added tags to movie data. Saved to: {output_file}")
display(result.head())

Added tags to movie data. Saved to: c:\Users\willi\Documents\GitHub\Movie-Recommendation-System\Cleaned Datasets\Final_Movie_Data.tsv


,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,directors,writers,tags
0,tt0000574,The Story of the Kelly Gang,0,1906,70,"Action,Adventure,Biography",6.0,nm0846879,nm0846879,"partially lost film, silent film"
1,tt0002130,Dante's Inferno,0,1911,71,"Adventure,Drama,Fantasy",7.1,"nm0078205,nm0655824,nm0209738",nm0019604,"youtube, based on song, poem or rhyme, hell"
2,tt0002423,Passion,0,1919,113,"Biography,Drama,Romance",6.7,nm0523932,"nm0266183,nm0473134",
3,tt0002844,Fantômas: In the Shadow of the Guillotine,0,1913,54,"Crime,Drama",6.9,nm0275421,"nm0019855,nm0275421,nm0816232",
4,tt0003014,Ingeborg Holm,0,1913,96,Drama,7.0,nm0803705,"nm0472236,nm0803705","poverty, workhouse, bankruptcy, broken heart, ..."


# Processing and Filtering Ratings Data with IMDb IDs

In this section, I will have to:
- remove the `timestamp` column as it is..
- normalise the `rating` from a 0-5 scale to a 0-1 scale, so that
- remove the rows of data where the movie isnt apart of the 

In [32]:
ratings_file = os.path.join(script_dir, "MovieLens Datasets", "ratings.csv")

df_ratings_raw = pd.read_csv(ratings_file)

# Remove the timestamp column
df_ratings = df_ratings_raw.drop(columns=["timestamp"])

# Normalise from 0-5 ratings to 0-1 
df_ratings["rating"] = df_ratings["rating"] / 5.0

display(df_ratings.head(5))

,userId,movieId,rating
0,1,1,0.8
1,1,110,0.8
2,1,158,0.8
3,1,260,0.9
4,1,356,1.0


Merging Ratings with IMDb Links

- The ratings dataset (`df_ratings`) is merged with the df_links dataset on movieId, to link ratings to IMDb IDs.
- Only the relevant columns (`userId`, `imdbId`, and `rating`) are kept.

The filtered dataset is then saved as ratings_imdb_matched.csv in the "Cleaned Datasets" folder, and the total number of ratings after filtering is displayed.

In [33]:
# Merge ratings with links to get IMDb IDs
df_ratings_filtered = df_ratings.merge(df_links, on="movieId", how="inner")

# Keep only relevant columns 
df_ratings_filtered = df_ratings_filtered[["userId", "imdbId", "rating"]]

# Save the filtered ratings dataset
filtered_ratings_file = os.path.join(script_dir, "Cleaned Datasets", "Audience_Ratings.csv")
df_ratings_filtered.to_csv(filtered_ratings_file, index=False)

# Display saved file locations 
print(f"Filtered ratings dataset saved to: {filtered_ratings_file}")
print(f"Total ratings after filtering: {len(df_ratings_filtered)}")

display(df_ratings_filtered.head(5))

MemoryError: cannot allocate memory for array

In [ ]:
# Original ratings file
print(f"\nratings.csv raw:")
display(df_ratings_raw.head(5))

# Cleaned and Normalised rating file 
print(f"\n Timestamp column removed, ratings normalised:")
display(df_ratings.head(5))

# Final file, corresponds to Datasets\Audience_Ratings.csv
print(f"\n Replaced movieID with imdbID:")
display(df_ratings_filtered.head(5))